In [1]:
import pandas as pd
from IPython.display import display
pd.set_option('display.float_format', lambda x: '%.3f' % x)
          
f = open('q2_brewery_count_by_state_1984_2017.csv', 'r')
f2 = open('age_by_state_2016.csv', 'r')
f3 = open('race_by_state_2016.csv', 'r')
f4 = open('median_hii_by_state.csv', 'r')
f5 = open('craft_state_stats.csv', 'r')

dfb = pd.read_csv(f)
dfa = pd.read_csv(f2)
dfr = pd.read_csv(f3)
dfh = pd.read_csv(f4)
dfc = pd.read_csv(f5)

First I'll take a look at the data that has the number of breweries by state

In [2]:
dfb.head()

,STATE,1984,1985,1986,1987,1988,1989,1990,1991,1992,...,2009,2010,2011,2012,2013,2014,2015,2016,2017 (through 3/31/2017),2017 (through 6/30/2017)
0,AK,nan,nan,nan,nan,nan,nan,nan,3.000,4.000,...,18.000,20.000,23.000,25.000,25.000,28.000,35.000,36.000,40.000,41.000
1,AL,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,5.000,7.000,9.000,15.000,23.000,25.000,30.000,37.000,37.000,45.000
2,AR,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,5.000,5.000,7.000,13.000,18.000,23.000,29.000,34.000,36.000,40.000
3,AZ,nan,nan,nan,nan,nan,6.000,8.000,8.000,7.000,...,34.000,37.000,45.000,55.000,59.000,77.000,91.000,110.000,120.000,125.000
4,CA,9.000,12.000,15.000,23.000,39.000,50.000,67.000,76.000,93.000,...,340.000,358.000,401.000,472.000,528.000,654.000,788.000,927.000,981.000,1049.000


From the first file (number of breweries by state) I will select just the 2016 data which is all we need for this analysis

There's a category for 'Other' in the data but there are no breweries calssified for that geo in 2016 so I'll remove it

In [3]:
dfb = dfb[['STATE', '2016']]
dfb = dfb.fillna(0)
dfb = dfb[~dfb['STATE'].isin(['Other'])]

dfb.head()

,STATE,2016
0,AK,36.000
1,AL,37.000
2,AR,34.000
3,AZ,110.000
4,CA,927.000


Looking at the age, race and hhi data by state for 2016

In [4]:
display(dfa.head(3))
display(dfr.head(3))
display(dfh.head(3))

,Title: Population Distribution by Age | The Henry J. Kaiser Family Foundation,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Timeframe: 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Location,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,Total
2,United States,78150300,29814800,39736000,82072200,41324800,49273900,320372000


,Title: Population Distribution by Race/Ethnicity | The Henry J. Kaiser Family Foundation,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Timeframe: 2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Location,White,Black,Hispanic,Asian,American Indian/Alaska Native,Native Hawaiian/Other Pacific Islander,Two Or More Races,Total
2,United States,195453000,39256900,57670000,18364500,2390300,1043200,6194100,320372000


,Title: Median Annual Household Income | The Henry J. Kaiser Family Foundation,Unnamed: 1
0,Timeframe: 2016,NaN
1,Location,Median Annual Household Income
2,United States,"$59,039"


Preparing the age, race and hhi data. We need to use row index 1 as the header, remove unnecessary columns (United States, total column for race) and the notes at the bottom.

The total column for race does not equal the sum of all the race columns, as there is some missing data. So I'll use the total poulation form the age data to get the precentage by race.

In [5]:
#Age
#Replacing the header with the 3rd row
new_header = dfa.iloc[1] 
dfa = dfa[2:] 
dfa.columns = new_header

#Removing United States
dfa = dfa[1:] 

#Removing the notes at the bottom
dfa = dfa.dropna(how = 'any')

#Race
new_header = dfr.iloc[1] 
dfr = dfr[2:] 
dfr.columns = new_header
dfr = dfr[1:] 
dfr = dfr.iloc[:51 ,:] #another way to get rid of the notes at the bottom
dfr = dfr.replace({'<10000':float(0)}) #unique to the race data
dfr.iloc[:, 1:] = dfr.iloc[:, 1:].astype(float) #needed to convert the obj to a number before adding
dfr['total_race'] = dfr.iloc[:, 1:8].sum(axis=1)
dfr = dfr.drop('Total', 1) #Dropping the original total column

#Median hhi
new_header = dfh.iloc[1] 
dfh = dfh[2:] 
dfh.columns = new_header
dfh = dfh[1:] 
dfh = dfh.iloc[:51 ,:]
dfh['Median Annual Household Income'].replace('[\$,]', '', regex=True, inplace=True) #the values are stored as currency ($47,221) so converting those

display(dfa.head(3))
display(dfa.tail(3))
display(dfr.head(3))
display(dfr.tail(3))
display(dfh.head(3))
display(dfh.tail(3))


1,Location,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,Total
3,Alabama,1154700,450200,539200,1248500,680400,761000,4834100
4,Alaska,201500,67700,107900,165500,84400,83900,710800
5,Arizona,1715400,702200,745500,1760500,853500,1113200,6890200


1,Location,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,Total
51,West Virginia,398200,162200,186100,468700,263600,335200,1814100
52,Wisconsin,1396100,535500,642100,1415900,853900,922600,5766100
53,Wyoming,152600,46200,69300,139000,81100,83500,571700


1,Location,White,Black,Hispanic,Asian,American Indian/Alaska Native,Native Hawaiian/Other Pacific Islander,Two Or More Races,total_race
3,Alabama,3204100.000,1304100.000,193500.000,NaN,NaN,NaN,59200.000,4760900.000
4,Alaska,425300.000,19900.000,47200.000,35300.000,118000.000,15900.000,49200.000,710800.000
5,Arizona,3728700.000,305300.000,2376900.000,223300.000,138000.000,NaN,84500.000,6856700.000


1,Location,White,Black,Hispanic,Asian,American Indian/Alaska Native,Native Hawaiian/Other Pacific Islander,Two Or More Races,total_race
51,West Virginia,1683500.000,62400.000,22000.000,NaN,NaN,0.000,NaN,1767900.000
52,Wisconsin,4614900.000,321700.000,481300.000,174300.000,NaN,0.000,121300.000,5713500.000
53,Wyoming,489400.000,4000.000,53900.000,NaN,NaN,NaN,11300.000,558600.000


1,Location,Median Annual Household Income
3,Alabama,47221
4,Alaska,75723
5,Arizona,57100


1,Location,Median Annual Household Income
51,West Virginia,44354
52,Wisconsin,59817
53,Wyoming,57829


The brewery data has the states as abbreviations and the age, race, hhi and craft beer consumption/production data has the state names so we need to change them so they match. For display purposes it's easier to have the abbreviations so we'll use those. Then I'll merge all three dataframes together.

In [6]:
#Source: https://gist.github.com/rogerallen/1583593
us_state_abbrev = {'Alabama': 'AL','Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR',
    'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE','District of Columbia': 'DC',
    'Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN',
    'Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD',
    'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO',
    'Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ','New Mexico': 'NM',
    'New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH','Oklahoma': 'OK','Oregon': 'OR',
    'Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC','South Dakota': 'SD','Tennessee': 'TN',
    'Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA','Washington': 'WA','West Virginia': 'WV',
    'Wisconsin': 'WI','Wyoming': 'WY',
}

dfa = dfa.replace({'Location': us_state_abbrev})
dfr = dfr.replace({'Location': us_state_abbrev})
dfh = dfh.replace({'Location': us_state_abbrev})

#Merging on the three dataframes on the location key
from functools import reduce
dfs = [dfa, dfr, dfh]
df_merged = reduce(lambda left, right: pd.merge(left, right, on='Location', how='inner'), dfs)

#Also converting the rest of the pandas objects to float and filling NaN with 0
df_merged.iloc[:, 1:] = df_merged.iloc[:, 1:].astype(float)
df_merged = df_merged.fillna(0)
df_merged

df_merged.head()

1,Location,Children 0-18,Adults 19-25,Adults 26-34,Adults 35-54,Adults 55-64,65+,Total,White,Black,Hispanic,Asian,American Indian/Alaska Native,Native Hawaiian/Other Pacific Islander,Two Or More Races,total_race,Median Annual Household Income
0,AL,1154700.000,450200.000,539200.000,1248500.000,680400.000,761000.000,4834100.000,3204100.000,1304100.000,193500.000,0.000,0.000,0.000,59200.000,4760900.000,47221.000
1,AK,201500.000,67700.000,107900.000,165500.000,84400.000,83900.000,710800.000,425300.000,19900.000,47200.000,35300.000,118000.000,15900.000,49200.000,710800.000,75723.000
2,AZ,1715400.000,702200.000,745500.000,1760500.000,853500.000,1113200.000,6890200.000,3728700.000,305300.000,2376900.000,223300.000,138000.000,0.000,84500.000,6856700.000,57100.000
3,AR,741500.000,269700.000,347600.000,743300.000,361100.000,482100.000,2945300.000,2134700.000,455100.000,206900.000,0.000,24500.000,0.000,64700.000,2885900.000,45907.000
4,CA,9677600.000,3855000.000,5035900.000,10377800.000,4834400.000,5407600.000,39188300.000,14848900.000,2115300.000,15125100.000,5787500.000,210700.000,248000.000,852600.000,39188100.000,66637.000


Looking at the craft beer produced (in bbls) and craft beer gallons consumed per adult by state for 2016 scraped from the Brewer's Association website

In [7]:
dfc.head()

,Unnamed: 0,craft_barr_produced,craft_gal_per_adult,state
0,0,"61,042",0.500,Alabama
1,1,"207,369",12.500,Alaska
2,2,"147,255",0.900,Arizona
3,3,"35,846",0.500,Arkansas
4,4,"3,295,221",3.700,California


In [8]:
#Checking the dtypes
dfc.dtypes

Unnamed: 0               int64
craft_barr_produced     object
craft_gal_per_adult    float64
state                   object
dtype: object

The data for 'craft_barr_produced' is an object so we need to convert it to a float

In [9]:
dfc['craft_barr_produced'] = dfc['craft_barr_produced'].str.replace(',','')
dfc['craft_barr_produced'] = dfc['craft_barr_produced'].astype(float)
dfc.dtypes

Unnamed: 0               int64
craft_barr_produced    float64
craft_gal_per_adult    float64
state                   object
dtype: object

In [10]:
dfc.head(3)

,Unnamed: 0,craft_barr_produced,craft_gal_per_adult,state
0,0,61042.000,0.500,Alabama
1,1,207369.000,12.500,Alaska
2,2,147255.000,0.900,Arizona


As before, I'll replace the state names with the state abbreviations

In [11]:
dfc = dfc.iloc[:, 1:]
dfc.rename(columns={'state':'Location'}, inplace=True)
dfc = dfc.replace({'Location': us_state_abbrev})
dfc.head()

,craft_barr_produced,craft_gal_per_adult,Location
0,61042.000,0.500,AL
1,207369.000,12.500,AK
2,147255.000,0.900,AZ
3,35846.000,0.500,AR
4,3295221.000,3.700,CA


Doing the final merge of all the dataframes by state and renaming the columns

In [12]:
df_b16 = dfb[['STATE', '2016']]
df_b16.columns = ['Location', '2016']

df_list = [df_b16, df_merged, dfc]
df_full = df_list[0]

for df_temp in df_list[1:]:
    df_full = df_full.merge(df_temp, on='Location')

df_full.columns = ['Location', 'b_count', 'kids_0_18', 'adults_19_25', 'adults_26_34', 'adults_35_54', 'adults_55_64', 'adults_over_65', 
                   'total_pop', 'white', 'black', 'hispanic', 'asian', 'amer_ind_ak', 'nat_haw_pac_is', 'two_or_more_r', 'total_race', 'median_hhi',
              'craft_barr_produced', 'craft_gal_per_adult']    
    
df_full.head()

,Location,b_count,kids_0_18,adults_19_25,adults_26_34,adults_35_54,adults_55_64,adults_over_65,total_pop,white,black,hispanic,asian,amer_ind_ak,nat_haw_pac_is,two_or_more_r,total_race,median_hhi,craft_barr_produced,craft_gal_per_adult
0,AK,36.000,201500.000,67700.000,107900.000,165500.000,84400.000,83900.000,710800.000,425300.000,19900.000,47200.000,35300.000,118000.000,15900.000,49200.000,710800.000,75723.000,207369.000,12.500
1,AL,37.000,1154700.000,450200.000,539200.000,1248500.000,680400.000,761000.000,4834100.000,3204100.000,1304100.000,193500.000,0.000,0.000,0.000,59200.000,4760900.000,47221.000,61042.000,0.500
2,AR,34.000,741500.000,269700.000,347600.000,743300.000,361100.000,482100.000,2945300.000,2134700.000,455100.000,206900.000,0.000,24500.000,0.000,64700.000,2885900.000,45907.000,35846.000,0.500
3,AZ,110.000,1715400.000,702200.000,745500.000,1760500.000,853500.000,1113200.000,6890200.000,3728700.000,305300.000,2376900.000,223300.000,138000.000,0.000,84500.000,6856700.000,57100.000,147255.000,0.900
4,CA,927.000,9677600.000,3855000.000,5035900.000,10377800.000,4834400.000,5407600.000,39188300.000,14848900.000,2115300.000,15125100.000,5787500.000,210700.000,248000.000,852600.000,39188100.000,66637.000,3295221.000,3.700


Our model will try and predict whether certain variables influence the number of breweries in a state. To account for the different poulations of each state, I'll use 'breweries per capita' (BPC) instead of the absolute
number of breweries in a state. BPC will be calculated for adults only. The age data we have is not able to be broken out by A21+. The closest we can get is A19+ so we'll use that.

In [13]:
df_full['total_adults'] = df_full['total_pop'] - df_full['kids_0_18']
df_full['b_per_capita'] = df_full['b_count']/df_full['total_adults']*100000
df_full.head()

,Location,b_count,kids_0_18,adults_19_25,adults_26_34,adults_35_54,adults_55_64,adults_over_65,total_pop,white,...,asian,amer_ind_ak,nat_haw_pac_is,two_or_more_r,total_race,median_hhi,craft_barr_produced,craft_gal_per_adult,total_adults,b_per_capita
0,AK,36.000,201500.000,67700.000,107900.000,165500.000,84400.000,83900.000,710800.000,425300.000,...,35300.000,118000.000,15900.000,49200.000,710800.000,75723.000,207369.000,12.500,509300.000,7.069
1,AL,37.000,1154700.000,450200.000,539200.000,1248500.000,680400.000,761000.000,4834100.000,3204100.000,...,0.000,0.000,0.000,59200.000,4760900.000,47221.000,61042.000,0.500,3679400.000,1.006
2,AR,34.000,741500.000,269700.000,347600.000,743300.000,361100.000,482100.000,2945300.000,2134700.000,...,0.000,24500.000,0.000,64700.000,2885900.000,45907.000,35846.000,0.500,2203800.000,1.543
3,AZ,110.000,1715400.000,702200.000,745500.000,1760500.000,853500.000,1113200.000,6890200.000,3728700.000,...,223300.000,138000.000,0.000,84500.000,6856700.000,57100.000,147255.000,0.900,5174800.000,2.126
4,CA,927.000,9677600.000,3855000.000,5035900.000,10377800.000,4834400.000,5407600.000,39188300.000,14848900.000,...,5787500.000,210700.000,248000.000,852600.000,39188100.000,66637.000,3295221.000,3.700,29510700.000,3.141


We'll also calculate the % of age and race for each state to also account for the different state populations

In [14]:
df_full.iloc[:, 2:8] = df_full.iloc[:, 2:8].apply(lambda x: x/df_full['total_pop'])
df_full.iloc[:, 9:16] = df_full.iloc[:, 9:16].apply(lambda x: x/df_full['total_race'])
df_full.head()

,Location,b_count,kids_0_18,adults_19_25,adults_26_34,adults_35_54,adults_55_64,adults_over_65,total_pop,white,...,asian,amer_ind_ak,nat_haw_pac_is,two_or_more_r,total_race,median_hhi,craft_barr_produced,craft_gal_per_adult,total_adults,b_per_capita
0,AK,36.000,0.283,0.095,0.152,0.233,0.119,0.118,710800.000,0.598,...,0.050,0.166,0.022,0.069,710800.000,75723.000,207369.000,12.500,509300.000,7.069
1,AL,37.000,0.239,0.093,0.112,0.258,0.141,0.157,4834100.000,0.673,...,0.000,0.000,0.000,0.012,4760900.000,47221.000,61042.000,0.500,3679400.000,1.006
2,AR,34.000,0.252,0.092,0.118,0.252,0.123,0.164,2945300.000,0.740,...,0.000,0.008,0.000,0.022,2885900.000,45907.000,35846.000,0.500,2203800.000,1.543
3,AZ,110.000,0.249,0.102,0.108,0.256,0.124,0.162,6890200.000,0.544,...,0.033,0.020,0.000,0.012,6856700.000,57100.000,147255.000,0.900,5174800.000,2.126
4,CA,927.000,0.247,0.098,0.129,0.265,0.123,0.138,39188300.000,0.379,...,0.148,0.005,0.006,0.022,39188100.000,66637.000,3295221.000,3.700,29510700.000,3.141


The data is now prepared and I'm ready to start analysing it. As a last step I'll just get rid of the columns we don't need anymore

In [15]:
df_full = df_full.drop(['b_count', 'total_pop', 'total_race', 'total_adults'], axis = 1)
df_full.head()

,Location,kids_0_18,adults_19_25,adults_26_34,adults_35_54,adults_55_64,adults_over_65,white,black,hispanic,asian,amer_ind_ak,nat_haw_pac_is,two_or_more_r,median_hhi,craft_barr_produced,craft_gal_per_adult,b_per_capita
0,AK,0.283,0.095,0.152,0.233,0.119,0.118,0.598,0.028,0.066,0.050,0.166,0.022,0.069,75723.000,207369.000,12.500,7.069
1,AL,0.239,0.093,0.112,0.258,0.141,0.157,0.673,0.274,0.041,0.000,0.000,0.000,0.012,47221.000,61042.000,0.500,1.006
2,AR,0.252,0.092,0.118,0.252,0.123,0.164,0.740,0.158,0.072,0.000,0.008,0.000,0.022,45907.000,35846.000,0.500,1.543
3,AZ,0.249,0.102,0.108,0.256,0.124,0.162,0.544,0.045,0.347,0.033,0.020,0.000,0.012,57100.000,147255.000,0.900,2.126
4,CA,0.247,0.098,0.129,0.265,0.123,0.138,0.379,0.054,0.386,0.148,0.005,0.006,0.022,66637.000,3295221.000,3.700,3.141


In [16]:
#Saving the data as a csv
df_full.to_csv('brew_data_prepped.csv')